In [ ]:
import numpy as np
import pandas as pd
from scipy import signal
import xarray as xr
import matplotlib.pyplot as plt
import dask
import xesmf as xe
import proplot as pplt
import cartopy.feature as cfeature
import xskillscore as xs

import warnings
warnings.filterwarnings("ignore")

In [ ]:
varname='precip'
model_types=['cnn_cat'] #,'logmodel_med']
#model_types=['logmodel_med']

# Define SEUS Region
lat_reg=[24.0,36.0]
lon_reg=[269.0,283.0]

# Define Winter and Summer Months
winter=[12,1,2]
summer=[6,7,8]

### Read in Precip and Calculate Anomalies

In [ ]:
files='/shared/ccsm4/khuang/obs/era-interim/era-interim.daily.precip.*.nc'
ds=xr.open_mfdataset(files,combine='by_coords')
ds_anoms=ds['precip'].groupby('time.dayofyear')-daily_climo(ds['precip'],'precip')


### Read in Predictor Fields

In [ ]:
zfiles='/shared/ccsm4/khuang/obs/era-interim/era-interim.daily.geoheight.*.nc'
ufiles='/shared/ccsm4/khuang/obs/era-interim/era-interim.daily.u.*.nc'
tfiles='/shared/ccsm4/khuang/obs/era-interim/era-interim.daily.sea.surface.temperature.0.75.x.0.75.1979-2018.nc'
olrfiles1='/shared/ccsm4/khuang/obs/era-interim/era-interim.daily.top.net.thermal.radiation.19*s.nc'
olrfiles2='/shared/ccsm4/khuang/obs/era-interim/era-interim.daily.top.net.thermal.radiation.20*s.nc'

In [ ]:
ds_z=xr.open_mfdataset(zfiles,combine='by_coords')
ds_z500=ds_z.sel(level=500).rename({'z':'z500'})
ds_z850=ds_z.sel(level=850).rename({'z':'z850'})

In [ ]:
ds_u=xr.open_mfdataset(ufiles,combine='by_coords')
ds_u200=ds_u.sel(level=200).rename({'u':'u200'})
ds_u850=ds_u.sel(level=850).rename({'u':'u850'})
ds_u250=ds_u.sel(level=250).rename({'u':'u250'})

In [ ]:
ds_sst=xr.open_mfdataset(tfiles,combine='by_coords')

In [ ]:
ds_olr1=xr.open_mfdataset(olrfiles1,combine='by_coords',combine_attrs='drop_conflicts')
ds_olr1['time']=ds_olr1['time'].dt.floor('d')
ds_olr2=xr.open_mfdataset(olrfiles2,combine='by_coords')
ds_olr=xr.combine_by_coords([ds_olr1,ds_olr2],combine_attrs='drop_conflicts').rename({'ttr':'olr'})

In [ ]:
new_lon=np.arange(0,360,5)
new_lat=np.arange(-90,92,5)

ds_out = xr.Dataset({'lat': (['lat'], new_lat),
                      'lon': (['lon'], new_lon)})

regridder = xe.Regridder(ds_anoms,ds_out,'bilinear',periodic=True)
ds_anoms=regridder(ds_anoms)

regridder = xe.Regridder(ds_z500,ds_out,'bilinear',periodic=True)
ds_z500re=regridder(ds_z500)
ds_z850re=regridder(ds_z850)
ds_u200re=regridder(ds_u200)
ds_u250re=regridder(ds_u250)
ds_u850re=regridder(ds_u850)
ds_olrre=regridder(ds_olr)

regridder_sst = xe.Regridder(ds_sst,ds_out,'bilinear',periodic=True)
ds_sstre=regridder_sst(ds_sst).fillna(0.0)